# 欠損値の扱い
欠損値を扱う上で、欠損値が発生する理由としては以下の３点が考えられます。
* 値が存在しないケース

例）個人と法人が混在しているデータでの法人の年齢、人数が０のデータの人数の平均値など

* 何らかの意図があるケース

入力側が意図的に入力していない。または観測していない場合。

* 値を取得するのに失敗したケース

人為的ミスや機器エラーで観測できなかった場合。

使用するモデルがGBDTの場合（xgboostやlightgbm）は、欠損値をそのまま扱うことができるので基本的には欠損値のまま扱います。

GBDTでないモデルの多くは欠損値を扱うことができずエラーとなるので、値を保管する必要があります。埋める方法として、代表値で埋める方法・他の変数から予測して埋める方法があります。

GBDTであっても、欠損を保管した方が精度が向上する場合があるので、両方を試してみるのも良いでしょう。また、欠損値から新たな特徴量を作成するのも有効です。

欠損値を含むレコードを除外する。または欠損値を含む変数を除外する方法もありますが。テストデータに欠損値が含まれる場合はこの方法は使用できません。

## 欠損値のまま扱う
GBDTライブラリでは欠損値をそのまま扱うことができます。欠損値が、何らかの理由を持って欠損していると考えると、その情報を捨てるのはもったいないため、そのまま取り扱うのが自然な方法です。

欠損値を扱うことのできない決定木ベースモデル（ランダムフォレストなど）においても、-9999など通常とる事のない値を代入する事で欠損値のまま取り扱うのに近い方法をとることができます。決定木は変数の値そのものではなく、変数の相対的な大小関係に依存してモデルが作成されるため、このようにすると欠損値か田舎でデータを分離できます。

## 代表値で埋める
欠損値を埋める方法として以下（数値変換の場合）

* 平均値

数値変換では最もオーソドックスな方法。分布が歪んでいるまたは外れ値が多い場合は対数変換などにより、歪みの少ない分布にしてから平均を取る方法もある。また、平均のとりかたも、全データの平均ではなく、別のカテゴリ変数の値でグループ分けし、そのグループごとの平均を代入する方法も考えられる。欠損している変数の分布がグループごとに大きく変わる場合に有効である、

* 中央値

 年収など、分布が歪んでいる場合、外れ値が多い場合は平均ではなく中央値を使用した方が良い
 
 カテゴリ変数の値ごとに平均を取る際に、データ数が極端に少ないカテゴリが存在する場合、その平均値にはあまり信用が置けませんし、そのカテゴリの値はすべて欠損しているかもしれません。その場合、分子と分母に定数項を足して計算させるBayesian averageという方法があります。

$$
\bar{x} = \frac{\sum_{i=1}^{n}x_i + Cm}{n + C}
$$

予め値mのデータをC個観測したことにして、平均の計算に加えています。データ数が少ない場合はmに、十分多い場合はそのカテゴリの平均に近づきます。mの値は事前知識として設定する必要がありますが、データ全体の平均値を用いるなどの方法で良いでしょう。

カテゴリ変数の場合は、欠損地を１つのカテゴリとみなして欠損を表すカテゴリを新たに作り置き変える方法や、最も多いカテゴリを代表値として置き換える方法があります。
 

In [4]:
from sklearn.datasets import fetch_openml

titanic = fetch_openml(data_id=40945, as_frame=True)
df1 = titanic.data
df2 = titanic.target
df = df1.join(df2)
df1.isnull().sum()

pclass          0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

## 欠損値を埋める

In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import codecs
%matplotlib inline

# 小数点第３位まで表示
%precision 3


list1 = [[10,20,30, 40],[np.nan, 25, 35, 45],[15, np.nan, 25, 35], [5, 15, np.nan, 30]]
columns = ['a', 'b', 'c', 'd']
df = pd.DataFrame(list1, columns=columns)


### サンプルデータ

In [24]:
df

,a,b,c,d
0,10.0,20.0,30.0,40
1,NaN,25.0,35.0,45
2,15.0,NaN,25.0,35
3,5.0,15.0,NaN,30


### 平均値で埋める

In [25]:
df.fillna(df.mean())

,a,b,c,d
0,10.0,20.0,30.0,40
1,10.0,25.0,35.0,45
2,15.0,20.0,25.0,35
3,5.0,15.0,30.0,30


### 中央値で埋める

In [28]:
df.fillna(df.median())

,a,b,c,d
0,10.0,20.0,30.0,40
1,10.0,25.0,35.0,45
2,15.0,20.0,25.0,35
3,5.0,15.0,30.0,30


### 最頻値で埋める

In [29]:
df.fillna(df.mode().iloc[0])

,a,b,c,d
0,10.0,20.0,30.0,40
1,5.0,25.0,35.0,45
2,15.0,15.0,25.0,35
3,5.0,15.0,25.0,30


### 特定の値で埋める

In [31]:
df.fillna(0)

,a,b,c,d
0,10.0,20.0,30.0,40
1,0.0,25.0,35.0,45
2,15.0,0.0,25.0,35
3,5.0,15.0,0.0,30


### 列ごとに異なる値で置換する

In [34]:
df.fillna({'a': '欠', 'b': '損', 'c': '直'})

,a,b,c,d
0,10,20,30,40
1,欠,25,35,45
2,15,損,25,35
3,5,15,直,30


### 欠損値が含まれる行を削除

In [52]:
df.dropna()

,a,b,c,d
0,10.0,20.0,30.0,40


In [50]:
# Baysian average
df2 = df.copy()

In [4]:
df2

,a,b,c,d
0,10.0,20.0,30.0,40
1,NaN,25.0,35.0,45
2,15.0,NaN,25.0,35
3,5.0,15.0,NaN,30


### Baysiean Average
Baysiean Averageを実行する関数です。第一引数に欠損値を補完したい列(例：df['a'])第二引数にあらかじめ入れたい値、第三引数に値の個数を入れて実行すると、補完後の列を返します。

In [46]:
def bay_avg(col, m, c):
    '''Bayesien Average
    Returns replaced column
    
    Parameters:
    --------------
    col : pandas.core.series.Series
            columns you want to replace (including missing data)
    m  :  int or float
            data you want to replace
    n   :  int
            numbers of m
    '''
    x1 = col.sum()
    n = col.shape[0] - col.isnull().sum()
    avg = x1 + (c * m) / (n + c)
    return col.fillna(avg)

サンプルデータにBaysiean Averageを適用した例
ここでは、サンプルデータのa列に値=100 が１０個入っていることにして平均を算出。欠損値を補完する

In [70]:
df2 = df.copy()
df['a'] =  bay_avg(df['a'], 100, 10)
df

,a,b,c,d
0,10.000000,20.0,30.0,40
1,106.923077,25.0,35.0,45
2,15.000000,NaN,25.0,35
3,5.000000,15.0,NaN,30


## 欠損値の抽出
### 特定の列の欠損値が含まれる列を表示

In [36]:
df[df['a'].isnull()]

,a,b,c,d
1,NaN,25.0,35.0,45


### 欠損値が含まれる行を表示

In [38]:
df[df.isnull().any(axis=1)]

,a,b,c,d
1,NaN,25.0,35.0,45
2,15.0,NaN,25.0,35
3,5.0,15.0,NaN,30


### 欠損値が含まれる列を表示

In [51]:
print(df.isnull().any())
df.loc[:, df.isnull().any()]

a     True
b     True
c     True
d    False
dtype: bool


,a,b,c
0,10.0,20.0,30.0
1,NaN,25.0,35.0
2,15.0,NaN,25.0
3,5.0,15.0,NaN
